# Устанавливаем библиотеки

In [ ]:
!pip install -q openai
!pip install -q tiktoken
!pip install -q python-docx
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 61.1 MB/s eta 0:00:00
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.0.0
    Uninstalling google-auth-oauthlib-1.0.0:
      Successfully uninstalled google-auth-oauthlib-1.0.0
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.1.0
    Uninstalling google-auth-httplib2-0.1.0:
      Successfully uninstalled google-auth-httplib2-0.1.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84

In [ ]:
import tiktoken
import getpass
import openai
import os
import io
import json
from googleapiclient.discovery import build
from google.colab import auth
from googleapiclient.http import MediaIoBaseDownload
from docx import Document

In [ ]:
# Ключ API пользователя и установка его как переменной окружения
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


# Настройка доступа к Google Диску и получение файлов из папки на Google Диске

In [ ]:
auth.authenticate_user()                                                        # Авторизации на доступ к Google Диску
drive_service = build('drive', 'v3')                                            # Создание объект service для доступа к Google Drive API

In [ ]:
folder_id = '1DeDUAiJmLwWsQHjS0DFcBzfC0oPhPhRH'                                 # Идентификатор папки, в которой находятся файлы
results = drive_service.files().list(q=f"'{folder_id}' in parents").execute()   # Список файлов из указанной папки на Google Диске
files = results.get('files', [])
files = [file for file in files if file['name'].endswith('.docx')]              # Удаляем файлы не имеющие расширение 'docx'

In [ ]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1dt0EXv_vbRplzOtF8W5JGSeLcPn2BdTH',
 'name': '12 сопроводительное письмо.docx'}

In [ ]:
texts = []

# Пройдем по списку файлов
for file in files:
    file_id = file['id']
    file_name = file['name']

    # Загрузим файл из Google Диска
    request = drive_service.files().get_media(fileId=file_id)
    file_io = io.BytesIO()
    downloader = MediaIoBaseDownload(file_io, request)
    done = False
    while done is False:
        _, done = downloader.next_chunk()

    # Прочитаем текст из файла docx
    document = Document(file_io)
    text = ""
    for paragraph in document.paragraphs:
        text += paragraph.text

    file['text'] = text

In [ ]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1dt0EXv_vbRplzOtF8W5JGSeLcPn2BdTH',
 'name': '12 сопроводительное письмо.docx',
 'text': 'Сопроводительное письмо — супер важное дополнение к любому резюме. Особенно если вы действительно заинтересованы в работе, как таковой, а не ставите эксперименты над эйчарами и просто хотите посмотреть, как много откликов соберете с резюме без cover letter.❓ Почему оно имеет значение: часто на джоб-сайтах (по типу hh.ru и superjob.ru) у эйчаров есть функция, которая позволяет отсеивать часть откликов на вакансию, если к резюме не приложено сопроводительное письмо. То есть, даже если вы будете мего крутым спецом, уважаемым сеньором с редким стеком, ваше резюме банально не дойдет до эйчара, если вы поленитесь написать хотя бы пару предложений о себе.Пример, как и что можно написать:«Добрый день.Я пишу на Python 4+ года. За это время занимался разработкой от маленьких скриптов до кру

# Считаем количество токенов в каждом ключе 'text' списка 'files'

In [ ]:
# Создаем экземпляр токенизатора
encoding = tiktoken.get_encoding("cl100k_base")

# Проходим по каждому элементу списка
for file in files:
    token_count = len(encoding.encode(file['text']))                            # Токенизируем текст и считаем количество токенов
    file['token'] = token_count                                                 # Записываем количество токенов в ключ 'token'

    print("name:", file['name'])
    print('token:', file['token'],'\n' )

name: 12 сопроводительное письмо.docx
token: 736 

name: 11 сопроводительное письмо.docx
token: 2854 

name: 10 сопроводительное письмо.docx
token: 431 

name: 9 сопроводительное писмо.docx
token: 4638 

name: 8 сопроводительное письмо.docx
token: 3094 

name: 6 сопроводительное письмо.docx
token: 2321 

name: 7 сопроводительное письмо.docx
token: 2091 

name: 5 сопровоительное письмо.docx
token: 3000 

name: 4 сопроводительное письмо.docx
token: 1633 

name: 2 сопроводительное письмо.docx
token: 3072 

name: 1 сопроводительное письмо.docx
token: 2379 

name: 3 сопроводительное письмо.docx
token: 4471 



In [ ]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1dt0EXv_vbRplzOtF8W5JGSeLcPn2BdTH',
 'name': '12 сопроводительное письмо.docx',
 'text': 'Сопроводительное письмо — супер важное дополнение к любому резюме. Особенно если вы действительно заинтересованы в работе, как таковой, а не ставите эксперименты над эйчарами и просто хотите посмотреть, как много откликов соберете с резюме без cover letter.❓ Почему оно имеет значение: часто на джоб-сайтах (по типу hh.ru и superjob.ru) у эйчаров есть функция, которая позволяет отсеивать часть откликов на вакансию, если к резюме не приложено сопроводительное письмо. То есть, даже если вы будете мего крутым спецом, уважаемым сеньором с редким стеком, ваше резюме банально не дойдет до эйчара, если вы поленитесь написать хотя бы пару предложений о себе.Пример, как и что можно написать:«Добрый день.Я пишу на Python 4+ года. За это время занимался разработкой от маленьких скриптов до кру

# Антотации из инструкций по написанию резюме



In [ ]:
system_cont = '''
Ты - эксперт и HR-специалист, профессионально составляющий сопроводительные письма для своих клиентов. Ты отлично структуририруешь текст и выделяешь ключевые моменты.
Возьми на себя роль редактора, которому предоставили текст для получения анотации, и сконцентрируйся только на его содержании, игнорируя вопросы форматирования и дизайна.
В своем анотации выдели из полученного текста основные идеи и требования.
Важно, чтобы анотация содержала список от 3 до 5 пунктов. Пронумеруй все пункты списка.
Выбери только информацию относящуюся к написанию сопроводительного письма, о резюме не пиши.
Каждый пункт списка анотации обязательно должен состоять из 3 или 4 предложений. Первое предложение содержит тезис пункта, а следующие предложения раскрывают этот тезис.
Оставь только самые важные пункты по содержанию сопроводительные письма.
'''

Так, как у части инструкций текст больше 4к токенов, то выбирам модель  'gpt-3.5-turbo-16k'

In [ ]:
# Проходим по каждому элементу списка
for file in files:
    messages = [{"role": "system", "content": system_cont},
                {"role": "user", "content": file['text']}]                     # f"{file['text']}"}

    completion = openai.ChatCompletion.create(
                  model="gpt-3.5-turbo-16k",
                  messages=messages,
                  temperature=0
                  )
    file['annotation'] = completion['choices'][0]['message']['content']

    print("name:", file['name'])
    print('annotation:', file['annotation'],'\n' )

name: 12 сопроводительное письмо.docx
annotation: 1. Сопроводительное письмо является важным дополнением к резюме и может повлиять на отбор кандидатов. Без него, даже если вы имеете выдающиеся навыки и опыт, ваше резюме может быть проигнорировано. 
2. В сопроводительном письме необходимо описать свой опыт работы и достижения, чтобы привлечь внимание работодателя. Укажите, чем вы занимались на предыдущем месте работы и какие результаты достигли. 
3. Важно также подчеркнуть свое стремление к развитию и повышению своих навыков. Укажите, как вы активно участвуете в мероприятиях и митапах, связанных с вашей профессией, а также о других способах самообразования. 
4. Сопроводительное письмо должно быть лаконичным и содержать от 500 до 1000 знаков. Важно начать с приветствия, затем описать свой опыт и достижения, а также подчеркнуть свое стремление к развитию. Закончите письмо заключением. 
5. Если вам сложно самостоятельно написать сопроводительное письмо, вы можете воспользоваться помощью Ch

In [ ]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1dt0EXv_vbRplzOtF8W5JGSeLcPn2BdTH',
 'name': '12 сопроводительное письмо.docx',
 'text': 'Сопроводительное письмо — супер важное дополнение к любому резюме. Особенно если вы действительно заинтересованы в работе, как таковой, а не ставите эксперименты над эйчарами и просто хотите посмотреть, как много откликов соберете с резюме без cover letter.❓ Почему оно имеет значение: часто на джоб-сайтах (по типу hh.ru и superjob.ru) у эйчаров есть функция, которая позволяет отсеивать часть откликов на вакансию, если к резюме не приложено сопроводительное письмо. То есть, даже если вы будете мего крутым спецом, уважаемым сеньором с редким стеком, ваше резюме банально не дойдет до эйчара, если вы поленитесь написать хотя бы пару предложений о себе.Пример, как и что можно написать:«Добрый день.Я пишу на Python 4+ года. За это время занимался разработкой от маленьких скриптов до кру

## Считаем количество токенов в анотациях

In [ ]:
total_tokens = 0

# Проходим по каждому элементу списка
for file in files:
    token_count = len(encoding.encode(file['annotation']))                      # Токенизируем анотацию и сосчитаем количество токенов
    file['token_anot'] = token_count                                            # Записываем количество токенов в ключ 'token'
    total_tokens += token_count
    print("name:", file['name'])
    print('token:', file['token_anot'],'\n' )

print("total_tokens =", total_tokens)

name: 12 сопроводительное письмо.docx
token: 461 

name: 11 сопроводительное письмо.docx
token: 386 

name: 10 сопроводительное письмо.docx
token: 312 

name: 9 сопроводительное писмо.docx
token: 430 

name: 8 сопроводительное письмо.docx
token: 312 

name: 6 сопроводительное письмо.docx
token: 401 

name: 7 сопроводительное письмо.docx
token: 449 

name: 5 сопровоительное письмо.docx
token: 557 

name: 4 сопроводительное письмо.docx
token: 699 

name: 2 сопроводительное письмо.docx
token: 433 

name: 1 сопроводительное письмо.docx
token: 508 

name: 3 сопроводительное письмо.docx
token: 520 

total_tokens = 5468


In [ ]:
files[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1dt0EXv_vbRplzOtF8W5JGSeLcPn2BdTH',
 'name': '12 сопроводительное письмо.docx',
 'text': 'Сопроводительное письмо — супер важное дополнение к любому резюме. Особенно если вы действительно заинтересованы в работе, как таковой, а не ставите эксперименты над эйчарами и просто хотите посмотреть, как много откликов соберете с резюме без cover letter.❓ Почему оно имеет значение: часто на джоб-сайтах (по типу hh.ru и superjob.ru) у эйчаров есть функция, которая позволяет отсеивать часть откликов на вакансию, если к резюме не приложено сопроводительное письмо. То есть, даже если вы будете мего крутым спецом, уважаемым сеньором с редким стеком, ваше резюме банально не дойдет до эйчара, если вы поленитесь написать хотя бы пару предложений о себе.Пример, как и что можно написать:«Добрый день.Я пишу на Python 4+ года. За это время занимался разработкой от маленьких скриптов до кру

# Попытка №2

In [ ]:
files_2 = files.copy()

In [ ]:
system_cont = '''
Ты профессиональный копирайтер. У тебя большой опыт работы с Бизнесом в разных сферах.
Ты профессионально составляешь и структурируешь тексты, умеешь выделять важные моменты в каждом тексте.
Твоя цель: Качественно структурировать текст на Русском языке и выделять основные Темы и Подтемы.
Оставь только самые важные пункты по содержанию сопроводительного письма.
'''

In [ ]:
# Проходим по каждому элементу списка
for file in files_2:
    messages = [{"role": "system", "content": system_cont},
                {"role": "user", "content": file['text']}]                     # f"{file['text']}"}

    completion = openai.ChatCompletion.create(
                  model="gpt-3.5-turbo-16k",
                  messages=messages,
                  temperature=0
                  )
    file['annotation'] = completion['choices'][0]['message']['content']

    print("name:", file['name'])
    print('annotation:', file['annotation'],'\n' )

name: 12 сопроводительное письмо.docx
annotation: - Сопроводительное письмо является важным дополнением к резюме.
- Отсутствие сопроводительного письма может привести к отсеиванию резюме на джоб-сайтах.
- В сопроводительном письме следует указать свой опыт работы и достижения.
- Важно также описать свое развитие в сфере и упомянуть участие в мероприятиях и проектах.
- Рекомендуется написать письмо объемом от 500 до 1000 знаков.
- В заключении можно добавить просьбу о проверке и корректировке сопроводительного письма ChatGPT. 

name: 11 сопроводительное письмо.docx
annotation: - Важно писать сопроводительные письма, которые отражают ценности и интересы работодателя, а не только себя.
- Не стоит описывать свои достижения и навыки, которые не имеют отношения к требованиям вакансии.
- Не следует задавать вопросы о зарплате и способе оплаты в сопроводительном письме.
- Важно подчеркнуть свои достижения и опыт, которые имеют прямое отношение к требованиям вакансии.
- Необходимо избегать грам

In [ ]:
files_2[0]

{'kind': 'drive#file',
 'mimeType': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
 'id': '1dt0EXv_vbRplzOtF8W5JGSeLcPn2BdTH',
 'name': '12 сопроводительное письмо.docx',
 'text': 'Сопроводительное письмо — супер важное дополнение к любому резюме. Особенно если вы действительно заинтересованы в работе, как таковой, а не ставите эксперименты над эйчарами и просто хотите посмотреть, как много откликов соберете с резюме без cover letter.❓ Почему оно имеет значение: часто на джоб-сайтах (по типу hh.ru и superjob.ru) у эйчаров есть функция, которая позволяет отсеивать часть откликов на вакансию, если к резюме не приложено сопроводительное письмо. То есть, даже если вы будете мего крутым спецом, уважаемым сеньором с редким стеком, ваше резюме банально не дойдет до эйчара, если вы поленитесь написать хотя бы пару предложений о себе.Пример, как и что можно написать:«Добрый день.Я пишу на Python 4+ года. За это время занимался разработкой от маленьких скриптов до кру

## Считаем количество токенов в анотациях

In [ ]:
total_tokens = 0

# Проходим по каждому элементу списка
for file in files_2:
    token_count = len(encoding.encode(file['annotation']))                      # Токенизируем анотацию и сосчитаем количество токенов
    file['token_anot'] = token_count                                            # Записываем количество токенов в ключ 'token'
    total_tokens += token_count
    print("name:", file['name'])
    print('token:', file['token_anot'],'\n' )

print("total_tokens =", total_tokens)

name: 12 сопроводительное письмо.docx
token: 215 

name: 11 сопроводительное письмо.docx
token: 275 

name: 10 сопроводительное письмо.docx
token: 127 

name: 9 сопроводительное писмо.docx
token: 363 

name: 8 сопроводительное письмо.docx
token: 250 

name: 6 сопроводительное письмо.docx
token: 322 

name: 7 сопроводительное письмо.docx
token: 234 

name: 5 сопровоительное письмо.docx
token: 365 

name: 4 сопроводительное письмо.docx
token: 134 

name: 2 сопроводительное письмо.docx
token: 444 

name: 1 сопроводительное письмо.docx
token: 876 

name: 3 сопроводительное письмо.docx
token: 178 

total_tokens = 3783


# Сумаризациия анотаций

## Объединяем все анотации одного примера в единый текст.

In [ ]:
annotations = ''

for file in files:
    annotations = annotations + ' ' + file['annotation']

## Сумаризируем сумму анотаций

In [ ]:
system_cont = '''
Ты профессиональный копирайтер. Ты профессионально составляешь и структурируешь тексты, умеешь выделять важные моменты в каждом тексте.
Необходимо объединить похожие по смыслу темы или подтемы. При необходимости перефразировать тему или подтему.
Надо откорректировать список, самые значимые темы поднять выше. Количество тем не должно превышать 15.
Каждый тема обязательно должена состоять из 3, 4 или 5 предложений. Первое предложение содержит тезис темы, а следующие предложения раскрывают этот тезис и дополняют его примерами.
'''

In [ ]:
messages = [{"role": "system", "content": system_cont},
            {"role": "user", "content": annotations}]
completion = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-16k",
              messages=messages,
              temperature=0
              )
summ_annotation = completion['choices'][0]['message']['content']

In [ ]:
summ_annotation

'- Сопроводительное письмо является важным дополнением к резюме и может повысить шансы на получение работы.\n- В письме следует указать свой опыт работы и достижения, которые соответствуют требованиям вакансии.\n- Важно подчеркнуть свои навыки и качества, которые делают вас подходящим кандидатом для данной должности.\n- Рекомендуется также упомянуть свое образование и профессиональное развитие.\n- Завершите письмо с призывом к действию и выражением готовности к собеседованию.'